In [4]:
import numpy as np
import pandas as pd
import os


In [8]:
description = pd.read_csv('data/patent_text_test_set_nlp_oct31_2022.csv')
description

,patent_id,patent_text
0,3930271,Golf glove A golf glove disclosed extra finger...
1,3930272,Crib leg lock A lock height-adjustable crib pl...
2,3930273,Bed safety side rail arrangement A bed safety ...
3,3930274,Assembly use recreational activities The assem...
4,3930276,Wheel spinning vehicle conveying apparatus aut...
...,...,...
99114,4035655,Method device implantation particles substrate...
99115,4035656,Method apparatus use approaching thermonuclear...
99116,4035657,Ozone generator An ozone generator air pump di...
99117,4035658,High power wind turbine kinetic accumulator A ...


In [ ]:
labels = pd.read_csv('data/patents_USPC_map_nlp_oct31_2022.csv')